# Домашнее задание 5. Обучение U-net

В этом задании вам предлагается обучить U-net для сегментации автомобилей. Подобная задача может быть актуальной для маркетплейсов, когда требуется отделить товар от фона, на котором он сфотографирован. Задание включает в себя:



1.   Загрузку и предобработку датасета из kaggle;
2.   Определение архитектуры U-net;
3.   Настройку пайплайна обучения.





## 1. Датасет

Мы воспользуеся датасетом [Carvana](https://www.kaggle.com/competitions/carvana-image-masking-challenge/overview), который уже содержит необходимые изображения и маски. Чтобы получить доступ к датасету, необходимо 1) зарегистрироваться на kaggle, 2) сгенерировать ключ, 3) вступить в соревнование, 4) загрузить датасет через kaggle API

In [1]:
# %pip install albumentations
# %pip install kagglehub
# %pip install ipywidgets

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

import shutil
from pathlib import Path
import json
import zipfile
import kagglehub
from tqdm import tqdm

In [3]:
with open("kaggle.json", "r") as file:
    api_token = json.load(file)

os.environ["KAGGLE_USERNAME"] = api_token["username"]
os.environ["KAGGLE_KEY"] = api_token["key"]

In [4]:
kagglehub.competition_download("carvana-image-masking-challenge", "train.zip")
kagglehub.competition_download("carvana-image-masking-challenge", "train_masks.zip")

100%|██████████| 405M/405M [00:37<00:00, 11.2MB/s] 


100%|██████████| 29.1M/29.1M [00:03<00:00, 7.86MB/s]


'C:\\Users\\ivana\\.cache\\kagglehub\\competitions\\carvana-image-masking-challenge\\train_masks.zip'

In [5]:
!move "C:\Users\ivana\.cache\kagglehub\competitions\carvana-image-masking-challenge\train_masks.zip" .
!move "C:\Users\ivana\.cache\kagglehub\competitions\carvana-image-masking-challenge\train.zip" .

��६�饭� 䠩���:         1.
��६�饭� 䠩���:         1.


In [6]:
with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile("train_masks.zip", 'r') as zip_ref:
    zip_ref.extractall()

Разделим данные на tran, val, test в соотношении 70:20:10.

In [7]:
# Задание исходных путей
image_dir = Path("train")
mask_dir = Path("train_masks")

# Целевые папки
splits = ["train", "val", "test"]
base_output_img = Path("images")
base_output_mask = Path("masks")

# Создание целевых директорий
for split in splits:
    (base_output_img / split).mkdir(parents=True, exist_ok=True)
    (base_output_mask / split).mkdir(parents=True, exist_ok=True)

# Получение списка всех изображений
image_files = [f for f in image_dir.glob("*.jpg")]
random.seed(777)
random.shuffle(image_files)

# Разделение на train/val/test
total = len(image_files)
n_train = int(total * 0.7)
n_val = int(total * 0.2)

train_files = image_files[:n_train]
val_files = image_files[n_train : n_train + n_val]
test_files = image_files[n_train + n_val :]


# Функция копирования изображений и соответствующих масок
def copy_files(file_list, split):
    for img_path in file_list:
        mask_name = img_path.stem + "_mask.gif"
        mask_path = mask_dir / mask_name

        # Копирование изображения
        shutil.copy(img_path, base_output_img / split / img_path.name)

        # Копирование маски
        if mask_path.exists():
            shutil.copy(mask_path, base_output_mask / split / mask_name)
        else:
            print(f"Маска не найдена: {mask_path}")


# Копирование файлов
copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print("Разделение завершено!")

Разделение завершено!


### 1.2 Аугментации через albumentations

В обучении U-net большую роль играет аугментация данных. Широкий набор нестандартных операций реализованы в библиотеке [albumentations](https://albumentations.ai/docs/2-core-concepts/transforms/). Вы можете ознакомиться с ней и добавить понравившиеся трансформации ниже в функцию *get_train_transform*

In [8]:
def get_train_transform():
    return A.Compose(
        [
            A.Resize(512, 512),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.HueSaturationValue(p=0.2),
            A.Normalize(
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
            ),
            ToTensorV2(),
        ]
    )


def get_val_transform():
    return A.Compose(
        [
            A.Resize(512, 512),
            A.Normalize(
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
            ),
            ToTensorV2(),
        ]
    )

In [9]:
class CarvanaDataset(Dataset):
    def __init__(
        self, images_dir, masks_dir, transform=None
    ):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_files = [
            f for f in os.listdir(images_dir) if f.lower().endswith(".jpg")
        ]
        random.shuffle(self.image_files)  # Перемешиваем файлы для обучения

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.images_dir, image_file)
        mask_path = os.path.join(
            self.masks_dir, image_file.replace(".jpg", "_mask.gif")
        )

        # Чтение и конвертация изображений через PIL
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # режим "L" — одноканальная маска

        # Перевод в numpy-массивы
        image_np = np.array(image)
        mask_np = np.array(mask) / 255.0

        if self.transform:
            augmented = self.transform(image=image_np, mask=mask_np)
            image_tensor = augmented["image"]
            mask_tensor = augmented["mask"]
            mask_tensor = mask_tensor.unsqueeze(0)  # Добавляем размерность для маски
        else:
            # Если нет трансформаций, добавим каналы и переведём в тензор вручную
            image_tensor = torch.from_numpy(image_np).permute(2, 0, 1).float() / 255.0
            mask_tensor = torch.from_numpy(mask_np).unsqueeze(0).float()

        return image_tensor, mask_tensor
    

## 2. Определение U-net модели

На этом шаге от вас требуется определить структуру U-net, как, например, на картинке ниже. Для этого вам помогут операции и слои:


*   nn.Conv2d - обычная свертка
*   nn.MaxPool2d - max-pool, для уменьшения размерности activation map-ов
*   nn.ConvTranspose2d - обратная свертка
*   F.relu - функция активации
*   torch.cat - конкатинация векторов в skip connection


![img](https://lh6.googleusercontent.com/Rx30jfXZqXnWX8CjmBaztePGMtCydUyeR_D6o1o-2kVnyg2cX-yyEmwYheeWJR2vxEAepYromNrriGyLeGuZatztKdCYCmiIrsSspW75EX9WxvOivLPKxfwkIvQji9MzJIHK0y5V)

In [10]:
class DoubleConv(nn.Module):
    """Две свёртки 3x3 с BatchNorm и ReLU"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size=3, padding="same", bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels, out_channels, kernel_size=3, padding="same", bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)


class Down(nn.Module):
    """Понижение разрешения: MaxPool + DoubleConv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels),
        )

    def forward(self, x):
        return self.block(x)


class Up(nn.Module):
    """Повышение разрешения: ConvTranspose2d + DoubleConv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(
            in_channels, in_channels // 2, kernel_size=2, stride=2
        )
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class UNet(nn.Module):
    """U-Net для сегментации изображений"""

    def __init__(self, in_channels, out_classes):
        super().__init__()
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)

        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)

        self.outc = nn.Conv2d(64, out_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        return self.outc(x)

## 3. Обучение U-net

In [11]:
def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    bar = tqdm(data_loader)

    for images, masks in bar:
        images = images.to(device).float()
        masks = masks.to(device).float()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        bar.set_description(f"Loss: {loss.item()}")

    return epoch_loss / len(data_loader)


def eval_model(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device).float()
            masks = masks.to(device).float()

            outputs = model(images)
            loss = criterion(outputs, masks)

            epoch_loss += loss.item()

    return epoch_loss / len(data_loader)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=3, out_classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
train_dataset = CarvanaDataset("images/train", "masks/train", transform=get_train_transform())
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = CarvanaDataset("images/val", "masks/val", transform=get_val_transform())
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

test_dataset = CarvanaDataset("images/test", "masks/test", transform=get_val_transform())
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [14]:
num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = eval_model(model, val_loader, criterion, device)

    print(
        f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}"
    )

Loss: 0.03068634122610092: 100%|██████████| 446/446 [11:12<00:00,  1.51s/it] 


Epoch 1/5, Train Loss: 0.0879, Val Loss: 0.0309


Loss: 0.06102577969431877: 100%|██████████| 446/446 [10:35<00:00,  1.42s/it] 


Epoch 2/5, Train Loss: 0.0291, Val Loss: 0.0250


Loss: 0.01909356378018856: 100%|██████████| 446/446 [10:34<00:00,  1.42s/it] 


Epoch 3/5, Train Loss: 0.0218, Val Loss: 0.0145


Loss: 0.030759073793888092: 100%|██████████| 446/446 [10:29<00:00,  1.41s/it]


Epoch 4/5, Train Loss: 0.0191, Val Loss: 0.0209


Loss: 0.012355638667941093: 100%|██████████| 446/446 [09:37<00:00,  1.29s/it]


Epoch 5/5, Train Loss: 0.0168, Val Loss: 0.0199


## 4. Оценка модели


В качестве метрики, оценивающей модель, воспользуемся [коэффициентом  Сёренсена](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D0%A1%D1%91%D1%80%D0%B5%D0%BD%D1%81%D0%B5%D0%BD%D0%B0), или Dice coefficient. Коэффициент Сёренсена можно использовать для сравнения попиксельного соответствия между прогнозируемой сегментацией и соответствующей ей истинной информацией.

In [15]:
def dice_coeff(model, val_loader, eps=1e-6):
    model.eval()
    total_dice = 0
    total_samples = 0

    with torch.no_grad():
        for images, masks in tqdm(val_loader):
            images = images.to(device).float()
            masks = masks.to(device).float()

            outputs = model(images)
            outputs = torch.sigmoid(
                outputs
            )  # Применяем сигмоиду для получения вероятностей
            preds = (outputs > 0.5).float()  # Бинаризация предсказаний

            # Вычисляем коэффициент Dice для каждой пары предсказание-маска
            for pred, mask in zip(preds, masks):
                intersection = (pred * mask).sum()
                union = pred.sum() + mask.sum()
                dice = (2.0 * intersection + eps) / (
                    union + eps
                )  # Добавляем малое значение для избежания деления на ноль
                total_dice += dice.item()
                total_samples += 1

    return total_dice / total_samples if total_samples > 0 else 0


dice_value = dice_coeff(model, test_loader)
print(f"Dice Coefficient on Test Set: {dice_value:.4f}")

100%|██████████| 64/64 [00:42<00:00,  1.49it/s]

Dice Coefficient on Test Set: 0.9831


Выссокое значение коэффициента достигается за счет того, что в датасете большое количество схожих автомобилей, снятых с разных углов.